In [163]:
import os
import glob
import pickle
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import randint
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [ ]:
df_raw = pd.read_csv('data/Cleaned_Migrant_Fatalities_Data.csv')

df_raw

In [ ]:
df_raw.info()

In [ ]:
column_rename_map = {
    'Region of Incident': 'region_incident',
    'Incident Date': 'incident_date',
    'Incident Year': 'incident_year',
    'Month': 'month_name', # Temporarily keep for review, then drop
    'Number of Dead': 'num_dead',
    'Minimum Estimated Number of Missing': 'min_est_missing',
    'Total Number of Dead and Missing': 'total_dead_missing',
    'Number of Survivors': 'num_survivors',
    'Number of Females': 'num_females',
    'Number of Males': 'num_males',
    'Number of Children': 'num_children',
    'Country of Origin': 'country_origin',
    'Region of Origin': 'region_origin',
    'Cause of Death': 'cause_death',
    'Country of Incident': 'country_incident',
    'Migration Route': 'migration_route',
    'Coordinates': 'coordinates', # Temporarily keep for review, then drop
    'UNSD Geographical Grouping': 'unsd_geo_grouping', # Temporarily keep for review, then drop
    'Year': 'year_duplicate', # Temporarily keep for review, then drop
    'Month_Num': 'incident_month' # Renaming to incident_month as confirmed
}
df_raw = df_raw.rename(columns=column_rename_map)

In [ ]:
filtered_cols = [
    'region_incident', 'country_incident', 'incident_date', 'incident_year', 'incident_month',
    'region_origin', 'country_origin', 'cause_death', 'migration_route',
    'min_est_missing', 'total_dead_missing', 'num_survivors','num_females', 'num_males', 'num_children', 'num_dead'
]

df_raw = df_raw[filtered_cols].copy()

In [ ]:
df_raw.isnull().sum()

In [ ]:
df_raw['incident_date'] = pd.to_datetime(df_raw['incident_date'], format='%Y-%m-%d')

In [ ]:
original_rows_before_date_drop = df_raw.shape[0]
df_raw.dropna(subset=['incident_date'], inplace=True)
print(f"\nDropped {original_rows_before_date_drop - df_raw.shape[0]} rows due to null 'incident_date'.")

In [ ]:
df_raw['incident_year'] = df_raw['incident_year'].astype('Int64')
df_raw['incident_month'] = df_raw['incident_month'].astype('Int64')
df_raw['incident_week'] = df_raw['incident_date'].dt.isocalendar().week.astype(int)

df_raw.head()

In [111]:
df_weekly_aggregated = df_raw.groupby(['region_incident', 'incident_year', 'incident_week']).agg(
    # Sum of numerical columns
    total_dead_missing=('total_dead_missing', 'sum'),
    num_dead=('num_dead', 'sum'),
    min_est_missing=('min_est_missing', 'sum'),
    num_survivors=('num_survivors', 'sum'),
    num_females=('num_females', 'sum'),
    num_males=('num_males', 'sum'),
    num_children=('num_children', 'sum'),
    # Count of original incidents in each aggregated unit
    num_incidents=('incident_date', 'count'),

    # Aggregating categorical features: taking the most frequent value (mode)
    # dropna=False ensures NaN values are treated as a distinct category if they are the most frequent.
    # Handling cases where x.mode() might be empty (e.g., all NaNs in a group)
    most_freq_country_origin=('country_origin', lambda x: x.mode(dropna=False)[0] if not x.mode(dropna=False).empty else 'Unknown'),
    most_freq_region_origin=('region_origin', lambda x: x.mode(dropna=False)[0] if not x.mode(dropna=False).empty else 'Unknown'),
    most_freq_cause_death=('cause_death', lambda x: x.mode(dropna=False)[0] if not x.mode(dropna=False).empty else 'Unknown'),
    most_freq_country_incident=('country_incident', lambda x: x.mode(dropna=False)[0] if not x.mode(dropna=False).empty else 'Unknown'),
    most_freq_migration_route=('migration_route', lambda x: x.mode(dropna=False)[0] if not x.mode(dropna=False).empty else 'Unknown'),

    # # Aggregating categorical features: count of unique values in each aggregated unit
    # num_unique_country_origin=('country_origin', 'nunique'),
    # num_unique_region_origin=('region_origin', 'nunique'),
    # num_unique_cause_death=('cause_death', 'nunique'),
    # num_unique_country_incident=('country_incident', 'nunique'),
    # num_unique_migration_route=('migration_route', 'nunique')
).reset_index()

df_weekly_aggregated['incident_year'] = df_weekly_aggregated['incident_year'].astype(int)
df_weekly_aggregated['incident_week'] = df_weekly_aggregated['incident_week'].astype(int)

# print shape after aggregation
print(f"\nShape after weekly aggregation: {df_weekly_aggregated.shape}")

# print columns after aggregation
print("\n--- Columns after weekly aggregation ---")
print(df_weekly_aggregated.columns.tolist())

print(f"\nOriginal number of rows (after incident_date drop): {df_raw.shape[0]}")
print(f"Number of rows after weekly aggregation (weeks with incidents): {df_weekly_aggregated.shape[0]}")


Shape after weekly aggregation: (4293, 16)

--- Columns after weekly aggregation ---
['region_incident', 'incident_year', 'incident_week', 'total_dead_missing', 'num_dead', 'min_est_missing', 'num_survivors', 'num_females', 'num_males', 'num_children', 'num_incidents', 'most_freq_country_origin', 'most_freq_region_origin', 'most_freq_cause_death', 'most_freq_country_incident', 'most_freq_migration_route']

Original number of rows (after incident_date drop): 19212
Number of rows after weekly aggregation (weeks with incidents): 4293


In [112]:
unique_regions = df_weekly_aggregated['region_incident'].unique()

min_year = df_weekly_aggregated['incident_year'].min()
max_year = df_weekly_aggregated['incident_year'].max()

all_weekly_dates = []
for year in range(min_year, max_year + 1):
    # Determine the maximum week number for the given year (can be 52 or 53)
    max_week_in_year = pd.to_datetime(f'{year}-12-31').isocalendar().week
    for week in range(1, max_week_in_year + 1):
        all_weekly_dates.append({'incident_year': year, 'incident_week': week})
full_weekly_date_range_df = pd.DataFrame(all_weekly_dates)

full_weekly_time_series_df = pd.merge(
    pd.DataFrame({'region_incident': unique_regions}),
    full_weekly_date_range_df,
    how='cross'
)

full_weekly_time_series_df['incident_year'] = full_weekly_time_series_df['incident_year'].astype(int)
full_weekly_time_series_df['incident_week'] = full_weekly_time_series_df['incident_week'].astype(int)

df_expanded_weekly = pd.merge(
    full_weekly_time_series_df,
    df_weekly_aggregated,
    on=['region_incident', 'incident_year', 'incident_week'],
    how='left'
)

categorical_cols_to_fill_no_incidents = [
    'most_freq_country_origin', 'most_freq_region_origin', 'most_freq_cause_death',
    'most_freq_country_incident', 'most_freq_migration_route'
]

numerical_cols_to_fill_zero = [
    'total_dead_missing', 'num_dead', 'min_est_missing',
    'num_survivors', 'num_females', 'num_males',
    'num_children', 'num_incidents',
    # 'num_unique_country_origin', 'num_unique_region_origin',
    # 'num_unique_cause_death', 'num_unique_country_incident',
    # 'num_unique_migration_route'
]

# Fill numerical columns with 0
for col in numerical_cols_to_fill_zero:
    if col in df_expanded_weekly.columns:
        df_expanded_weekly[col] = df_expanded_weekly[col].fillna(0)

# Fill categorical columns with 'No Incidents'
for col in categorical_cols_to_fill_no_incidents:
    if col in df_expanded_weekly.columns:
        df_expanded_weekly[col] = df_expanded_weekly[col].fillna('No Incidents')

# Sort the expanded DataFrame for correct lag/rolling calculations later
df_expanded_weekly = df_expanded_weekly.sort_values(
    by=['region_incident', 'incident_year', 'incident_week']
).reset_index(drop=True)

print(f"\nShape after filling NaNs: {df_expanded_weekly.shape}")

print("\n--- df_expanded_weekly Info after filling NaNs ---")
print(df_expanded_weekly.info())


Shape after filling NaNs: (7049, 16)

--- df_expanded_weekly Info after filling NaNs ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7049 entries, 0 to 7048
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   region_incident             7049 non-null   object 
 1   incident_year               7049 non-null   int64  
 2   incident_week               7049 non-null   int64  
 3   total_dead_missing          7049 non-null   float64
 4   num_dead                    7049 non-null   float64
 5   min_est_missing             7049 non-null   float64
 6   num_survivors               7049 non-null   float64
 7   num_females                 7049 non-null   float64
 8   num_males                   7049 non-null   float64
 9   num_children                7049 non-null   float64
 10  num_incidents               7049 non-null   float64
 11  most_freq_country_origin    7049 non-null   object 
 12  

In [113]:
# Recalculate the 75th percentile threshold on the expanded weekly data
weekly_high_fatality_threshold = df_expanded_weekly['total_dead_missing'].quantile(0.75)
print(f"Recalculated 75th percentile threshold for 'total_dead_missing' (weekly expanded data) is: {weekly_high_fatality_threshold:.2f}")

# Create the 'is_high_risk' target variable
df_expanded_weekly['is_high_risk'] = (df_expanded_weekly['total_dead_missing'] >= weekly_high_fatality_threshold).astype(int)

# Check the distribution of the new target variable
print("\n--- Distribution of 'is_high_risk' target variable (Weekly) ---")
print(df_expanded_weekly['is_high_risk'].value_counts().to_string())

print("\n--- Percentage distribution of 'is_high_risk' target variable (Weekly) ---")
print(df_expanded_weekly['is_high_risk'].value_counts(normalize=True).to_string())

Recalculated 75th percentile threshold for 'total_dead_missing' (weekly expanded data) is: 3.00

--- Distribution of 'is_high_risk' target variable (Weekly) ---
is_high_risk
0    5062
1    1987

--- Percentage distribution of 'is_high_risk' target variable (Weekly) ---
is_high_risk
0    0.718116
1    0.281884


In [114]:
# Numerical columns for which to create lag and rolling features
# Focus only on migration activity patterns, not fatality outcomes
numerical_cols_for_lags_and_rolls = [
    'num_survivors',     # Rescue/safety patterns (not direct fatality measure)
    'num_incidents',     # Migration activity level
    'num_females',       # Demographics patterns
    'num_males',         # Demographics patterns
    'num_children'       # Vulnerable population patterns
]

# Loop through selected numerical columns to create lag and rolling features
for col in numerical_cols_for_lags_and_rolls:
    # Create Lag Features (1-week lag)
    df_expanded_weekly[f'{col}_lag1'] = df_expanded_weekly.groupby('region_incident')[col].shift(1)
    df_expanded_weekly[f'{col}_lag2'] = df_expanded_weekly.groupby('region_incident')[col].shift(2)
    df_expanded_weekly[f'{col}_lag3'] = df_expanded_weekly.groupby('region_incident')[col].shift(3)

    # Create Rolling Window Features (4-week average for monthly trends)
    # df_expanded_weekly[f'{col}_rolling4w_emwa'] = df_expanded_weekly.groupby('region_incident')[col].transform(
    #     lambda x: x.ewm(span=4, min_periods=1).mean()
    # )

# # Create lag/rolling features for route diversity (indicates migration pressure)
# df_expanded_weekly['num_unique_migration_route_lag1'] = df_expanded_weekly.groupby('region_incident')['num_unique_migration_route'].shift(1)
# df_expanded_weekly['num_unique_migration_route_rolling4w_emwa'] = df_expanded_weekly.groupby('region_incident')['num_unique_migration_route'].transform(
#     lambda x: x.rolling(window=4, min_periods=1).mean()
# )

# Fill NaNs created by lags/rolling windows
lag_rolling_cols = [col for col in df_expanded_weekly.columns if '_lag' in col or '_rolling4w_emwa' in col]
for col in lag_rolling_cols:
    df_expanded_weekly[col] = df_expanded_weekly[col].fillna(0)

print(f"\n--- Created {len(lag_rolling_cols)} lag/rolling features ---")
print(f"Features created: {lag_rolling_cols}")

print("\n--- df_expanded_weekly Info after Reduced Lag and Rolling Features ---")
print(df_expanded_weekly.info())

print("\n--- First 5 rows of after adding temporal features ---")
print(df_expanded_weekly.head().to_string(index=False))


--- Created 15 lag/rolling features ---
Features created: ['num_survivors_lag1', 'num_survivors_lag2', 'num_survivors_lag3', 'num_incidents_lag1', 'num_incidents_lag2', 'num_incidents_lag3', 'num_females_lag1', 'num_females_lag2', 'num_females_lag3', 'num_males_lag1', 'num_males_lag2', 'num_males_lag3', 'num_children_lag1', 'num_children_lag2', 'num_children_lag3']

--- df_expanded_weekly Info after Reduced Lag and Rolling Features ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7049 entries, 0 to 7048
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   region_incident             7049 non-null   object 
 1   incident_year               7049 non-null   int64  
 2   incident_week               7049 non-null   int64  
 3   total_dead_missing          7049 non-null   float64
 4   num_dead                    7049 non-null   float64
 5   min_est_missing             7049 non-null   floa

In [115]:
# Create cyclical features for 'incident_week' (replacing incident_month_sin/cos)
max_week_for_cycle = df_expanded_weekly['incident_week'].max() # Will be 52 or 53
df_expanded_weekly['incident_week_sin'] = np.sin(2 * np.pi * df_expanded_weekly['incident_week'] / max_week_for_cycle)
df_expanded_weekly['incident_week_cos'] = np.cos(2 * np.pi * df_expanded_weekly['incident_week'] / max_week_for_cycle)

print("\n--- df_expanded_weekly Info after adding pct_change and cyclical week features ---")
print(df_expanded_weekly.info())

print("\n--- First 10 rows of df_expanded_weekly with new temporal features ---")
print(df_expanded_weekly.head(10).to_string(index=False))


--- df_expanded_weekly Info after adding pct_change and cyclical week features ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7049 entries, 0 to 7048
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   region_incident             7049 non-null   object 
 1   incident_year               7049 non-null   int64  
 2   incident_week               7049 non-null   int64  
 3   total_dead_missing          7049 non-null   float64
 4   num_dead                    7049 non-null   float64
 5   min_est_missing             7049 non-null   float64
 6   num_survivors               7049 non-null   float64
 7   num_females                 7049 non-null   float64
 8   num_males                   7049 non-null   float64
 9   num_children                7049 non-null   float64
 10  num_incidents               7049 non-null   float64
 11  most_freq_country_origin    7049 non-null   object 
 12  most_f

In [116]:
df_expanded_weekly.columns

Index(['region_incident', 'incident_year', 'incident_week',
       'total_dead_missing', 'num_dead', 'min_est_missing', 'num_survivors',
       'num_females', 'num_males', 'num_children', 'num_incidents',
       'most_freq_country_origin', 'most_freq_region_origin',
       'most_freq_cause_death', 'most_freq_country_incident',
       'most_freq_migration_route', 'is_high_risk', 'num_survivors_lag1',
       'num_survivors_lag2', 'num_survivors_lag3', 'num_incidents_lag1',
       'num_incidents_lag2', 'num_incidents_lag3', 'num_females_lag1',
       'num_females_lag2', 'num_females_lag3', 'num_males_lag1',
       'num_males_lag2', 'num_males_lag3', 'num_children_lag1',
       'num_children_lag2', 'num_children_lag3', 'incident_week_sin',
       'incident_week_cos'],
      dtype='object')

In [117]:
# Define features (X) and target (y)
# Exclude columns that directly represent the outcome or its components to prevent data leakage
# Also exclude incident_year from features as it's used for splitting and direct year can cause overfitting.
X = df_expanded_weekly.drop(columns=[
    'total_dead_missing',         # Directly used to create 'is_high_risk'
    'num_dead',                   # Component of total_dead_missing
    'min_est_missing',            # Component of total_dead_missing
    'num_females',                # Female casualties - LEAKAGE
    'num_males',                  # Male casualties - LEAKAGE
    'num_children',               # Child casualties - LEAKAGE
    'is_high_risk',               # The target variable itself
    'incident_week',              # Cyclical feature, not needed as raw week
    'incident_year'               # Used for splitting, typically not a direct feature in this form
])
y = df_expanded_weekly['is_high_risk']

# print X information
print("\n--- Features (X) Info ---")
print(X.info())


--- Features (X) Info ---


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7049 entries, 0 to 7048
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   region_incident             7049 non-null   object 
 1   num_survivors               7049 non-null   float64
 2   num_incidents               7049 non-null   float64
 3   most_freq_country_origin    7049 non-null   object 
 4   most_freq_region_origin     7049 non-null   object 
 5   most_freq_cause_death       7049 non-null   object 
 6   most_freq_country_incident  7049 non-null   object 
 7   most_freq_migration_route   7049 non-null   object 
 8   num_survivors_lag1          7049 non-null   float64
 9   num_survivors_lag2          7049 non-null   float64
 10  num_survivors_lag3          7049 non-null   float64
 11  num_incidents_lag1          7049 non-null   float64
 12  num_incidents_lag2          7049 non-null   float64
 13  num_incidents_lag3          7049 

In [118]:
split_year = 2023 # As decided earlier

# Perform time-series aware train-test split
# Ensure X and y are properly aligned using the index
train_indices = df_expanded_weekly['incident_year'] < split_year
test_indices = df_expanded_weekly['incident_year'] >= split_year

X_train = X[train_indices].copy()
X_test = X[test_indices].copy()
y_train = y[train_indices].copy()
y_test = y[test_indices].copy()

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# Verify the time ranges
print(f"\nX_train incident_year range: {df_expanded_weekly[train_indices]['incident_year'].min()} - {df_expanded_weekly[train_indices]['incident_year'].max()}")
print(f"X_test incident_year range: {df_expanded_weekly[test_indices]['incident_year'].min()} - {df_expanded_weekly[test_indices]['incident_year'].max()}")

Shape of X_train: (6023, 25)
Shape of X_test: (1026, 25)
Shape of y_train: (6023,)
Shape of y_test: (1026,)

X_train incident_year range: 2014 - 2022
X_test incident_year range: 2023 - 2025


In [119]:
# Identify categorical columns that need encoding (these are the 'most_freq' and region_incident)
# Ensure these columns exist in X_train/X_test
categorical_cols_for_encoding = [
    'region_incident',
    'most_freq_country_origin',
    'most_freq_region_origin',
    'most_freq_cause_death',
    'most_freq_country_incident',
    'most_freq_migration_route'
]

# Create a copy to avoid SettingWithCopyWarning during encoding
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

# Initialize TargetEncoder
# 'smoothing' and 'min_samples_leaf' help prevent overfitting by balancing
# the categorical average with the global average, especially important for rare categories.
encoder = TargetEncoder(cols=categorical_cols_for_encoding, smoothing=1.0, min_samples_leaf=1)

# Fit the encoder on the training data (X_train_encoded and y_train)
encoder.fit(X_train_encoded, y_train)

# Transform both training and test data
X_train_encoded = encoder.transform(X_train_encoded)
X_test_encoded = encoder.transform(X_test_encoded)

print("\n--- X_train_encoded head after Target Encoding ---")
print(X_train_encoded.head().to_string(index=False))

print("\n--- X_test_encoded head after Target Encoding ---")
print(X_test_encoded.head().to_string(index=False))

print("\n--- Data types in X_train_encoded after Target Encoding ---")
print(X_train_encoded.info())


--- X_train_encoded head after Target Encoding ---
 region_incident  num_survivors  num_incidents  most_freq_country_origin  most_freq_region_origin  most_freq_cause_death  most_freq_country_incident  most_freq_migration_route  num_survivors_lag1  num_survivors_lag2  num_survivors_lag3  num_incidents_lag1  num_incidents_lag2  num_incidents_lag3  num_females_lag1  num_females_lag2  num_females_lag3  num_males_lag1  num_males_lag2  num_males_lag3  num_children_lag1  num_children_lag2  num_children_lag3  incident_week_sin  incident_week_cos
        0.192429            0.0            0.0                  0.000000                 0.000000               0.000000                    0.000000                   0.052023                 0.0                 0.0                 0.0                 0.0                 0.0                 0.0               0.0               0.0               0.0             0.0             0.0             0.0                0.0                0.0                0.0 

In [120]:
# Apply SMOTE to the training data only

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_encoded, y_train)

print("\n--- Class distribution before SMOTE ---")
print(y_train.value_counts().to_string())
print("\n--- Class distribution after SMOTE ---")
print(y_train_resampled.value_counts().to_string())

print(f"\nShape of X_train_resampled: {X_train_resampled.shape}")
print(f"Shape of y_train_resampled: {y_train_resampled.shape}")


--- Class distribution before SMOTE ---
is_high_risk
0    4405
1    1618

--- Class distribution after SMOTE ---
is_high_risk
0    4405
1    4405

Shape of X_train_resampled: (8810, 25)
Shape of y_train_resampled: (8810,)


In [127]:
rf_baseline = RandomForestClassifier(n_estimators=50, random_state=42)

rf_baseline.fit(X_train_resampled, y_train_resampled)

y_pred = rf_baseline.predict(X_test_encoded)

print("\n--- Baseline Random Forest Classifier Performance ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

feature_importance = pd.DataFrame({
    'feature': X_train_resampled.columns,
    'importance': rf_baseline.feature_importances_ * 100
}).sort_values('importance', ascending=False)

print("\n--- Feature Importances from Random Forest ---")
print(feature_importance)


--- Baseline Random Forest Classifier Performance ---
Accuracy: 0.9337
Precision: 0.9216
Recall: 0.8916
F1 Score: 0.9063

--- Feature Importances from Random Forest ---
                       feature  importance
2                num_incidents   22.542262
6   most_freq_country_incident   18.476182
3     most_freq_country_origin   18.266835
5        most_freq_cause_death    9.179675
4      most_freq_region_origin    7.300511
7    most_freq_migration_route    5.161491
12          num_incidents_lag2    3.259012
0              region_incident    3.002721
19              num_males_lag3    2.563140
1                num_survivors    1.843414
13          num_incidents_lag3    1.522611
23           incident_week_sin    1.200492
18              num_males_lag2    1.130286
24           incident_week_cos    1.097923
11          num_incidents_lag1    0.866264
17              num_males_lag1    0.559698
8           num_survivors_lag1    0.350879
9           num_survivors_lag2    0.304411
10          n

In [159]:
top_k = 15
important_features = feature_importance.head(top_k)['feature'].tolist()

# find cumulative importance threshold from feature_importance
cumulative_threshold = feature_importance['cumulative_importance'].iloc[top_k - 1]

print(f"Selected {len(important_features)} features:")
print(feature_importance.head(top_k).to_string(index=False))

print(f"\nCumulative importance threshold for top {top_k} features: {cumulative_threshold:.2f}%")

Selected 15 features:
                   feature  importance  cumulative_importance
             num_incidents   22.542262              22.542262
most_freq_country_incident   18.476182              41.018445
  most_freq_country_origin   18.266835              59.285280
     most_freq_cause_death    9.179675              68.464955
   most_freq_region_origin    7.300511              75.765466
 most_freq_migration_route    5.161491              80.926957
        num_incidents_lag2    3.259012              84.185969
           region_incident    3.002721              87.188690
            num_males_lag3    2.563140              89.751830
             num_survivors    1.843414              91.595243
        num_incidents_lag3    1.522611              93.117854
         incident_week_sin    1.200492              94.318346
            num_males_lag2    1.130286              95.448633
         incident_week_cos    1.097923              96.546556
        num_incidents_lag1    0.866264          

In [160]:
X_train_reduced = X_train_resampled[important_features]
X_test_reduced = X_test_encoded[important_features]

print(f"\nShape of X_train_reduced: {X_train_reduced.shape}")
print(f"Shape of X_test_reduced: {X_test_reduced.shape}")


Shape of X_train_reduced: (8810, 15)
Shape of X_test_reduced: (1026, 15)


In [ ]:
# # enhance the param_grid into much more comprehensive search space
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 15, 20, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# tscv = TimeSeriesSplit(n_splits=5)
# rf_classifier = RandomForestClassifier(random_state=42)

# grid_search = GridSearchCV(
#     estimator=rf_classifier,
#     param_grid=param_grid,
#     scoring='f1',
#     cv=tscv,
#     n_jobs=1,
#     verbose=1,
#     return_train_score=True,
#     refit=True
# )

# grid_search.fit(X_train_reduced, y_train_resampled)

# print("\n--- Best Parameters from Grid Search ---")
# print(grid_search.best_params_)

# best_model = grid_search.best_estimator_

# timestamp = int(datetime.datetime.now().timestamp())

# with open(f'models/best_model_{timestamp}.pkl', 'wb') as f:
#     pickle.dump(best_model, f)


In [ ]:
# Define parameter distributions for RandomizedSearchCV
param_distributions = {
    'n_estimators': randint(50, 400),          # Random integers between 50-399
    'max_depth': [10, 15, 20, 25, 30, None],   # Discrete choices including None
    'min_samples_split': randint(2, 15),       # Random integers between 2-14
    'min_samples_leaf': randint(1, 8),         # Random integers between 1-7
    'max_features': ['sqrt', 'log2', None],    # Additional parameter for variety
}

tscv = TimeSeriesSplit(n_splits=5)
rf_classifier = RandomForestClassifier(random_state=42)

# RandomizedSearchCV with more parameter combinations but fewer total fits
random_search = RandomizedSearchCV(
    estimator=rf_classifier,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='f1',
    cv=tscv,
    n_jobs=1,
    verbose=1,
    random_state=42,
    return_train_score=True,
    refit=True
)

# Fit the random search
random_search.fit(X_train_reduced, y_train_resampled)

print("\n--- Best Parameters from Random Search ---")
print(random_search.best_params_)
print(f"\n--- Best CV Score: {random_search.best_score_:.4f} ---")

best_model = random_search.best_estimator_

# Save the model
timestamp = int(datetime.datetime.now().timestamp())
with open(f'models/best_model_{timestamp}.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [ ]:
# load from saved model with highest timestamp for each type of migrants

def load_best_model():
    model_files = glob.glob(f'models/best_model_*.pkl')

    if not model_files:
        raise FileNotFoundError(f"No saved models found.")

    # Sort the files by timestamp (filename) and get the latest one
    latest_model_file = max(model_files, key=os.path.getctime)

    with open(latest_model_file, 'rb') as f:
        print(f"Loading model from: {latest_model_file}")
        model = pickle.load(f)

    return model

best_model = load_best_model()

In [ ]:
# predict on the test set
y_pred = best_model.predict(X_test_reduced)

print("\n--- Best Model Performance on Test Set ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

In [ ]:
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 6))
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['Low Risk', 'High Risk'],
    yticklabels=['Low Risk', 'High Risk']
)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# plot feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance.head(top_k))
plt.title('Top 15 Feature Importances')
plt.xlabel('Importance (%)')
plt.ylabel('Feature')
plt.show()

print("\n--- Top 10 Features and Their Importance ---")
print(feature_importance.head(10).to_string(index=False))